In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')


class GPT2Dataset(Dataset):
    def __init__(self, questions, answers, tokenizer):
        self.questions = questions
        self.answers = answers
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        inputs = self.tokenizer(question, return_tensors='pt', truncation=True, padding='max_length', max_length=128)
        labels = self.tokenizer(answer, return_tensors='pt', truncation=True, padding='max_length', max_length=128)

        return inputs.input_ids.squeeze(), labels.input_ids.squeeze()

num_epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

questions = ["What are incentives allcoated","which pools are incentivized","what is tvl of all the pools"]
answers = ["50000","top 10","280M"]

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)
tokenizer.pad_token = tokenizer.eos_token

dataset = GPT2Dataset(questions, answers, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

optimizer = AdamW(model.parameters())

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        outputs = model(inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
def ask_question(model, question, tokenizer):
    # Tokenize the question
    inputs = tokenizer.encode(question + tokenizer.eos_token, return_tensors='pt').to(device)

    # Generate an answer
    outputs = model.generate(
        inputs,
        max_length=200,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        temperature=0.7,
    )

    # Decode the output
    answer = tokenizer.decode(outputs[0])
    
    return answer

# Use the model to answer a question
question = "What are the incentives allocated?"
answer = ask_question(model, question, tokenizer)
print(answer)


NameError: name 'Dataset' is not defined